# 네이버 플레이스 블로그 리뷰 크롤링
#### 2020-05-10 (일) 이한울
---
- 2020-05-10 (일): 플레이스 블로그 리뷰 긁어오기 및 스레딩 적용 완료
---
#### 2020-05-11 (월) 이한울
---
- 스레딩 실행 창에서 driver 경로를 본인의 크롬 드라이버가 있는 경로로 설정 후 실행
- time.sleep을 넉넉하게 주어야 HTTP 에러를 최대한 회피할 수 있습니다.
---
#### 2020-05-14 (목) 이한울
---
- 구글 드라이브 > 팀프로젝트 폴더 > [00]...마리톡 > [03] 데이터 > [03] 네이버 플레이스 리뷰 크롤링 > **naver_place_count_total_2020-05-01_final.csv** 파일을 다운로드, 원하는 경로로 이동시킨 뒤 진행해야 합니다.

In [38]:
import  urllib.request
import selenium
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from requests import ConnectionError, HTTPError, Timeout, TooManyRedirects
import urllib3

from pandas import Series, DataFrame
import pandas as pd
import numpy as np

from datetime import datetime
import time
import sys
import math
import re
from tqdm import tqdm, tqdm_notebook

import selenium.common.exceptions as sce
from selenium.webdriver.common.by import By
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium import webdriver

from queue import Queue
from multiprocessing.pool import ThreadPool
from threading import Thread
from multiprocessing import Pool
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

import glob
import fnmatch

In [39]:
# 블로그 검색 결과를 저장할 DataFrame 생성
df_blog_review = \
DataFrame(columns=['store_id', 'store_name', 'title', 'writer', 'link', 'summary', 'blog_name', 'original_index'])

In [40]:
df_blog_error = DataFrame(columns=['store_id','store_name','error_code', 'original_index'])

In [41]:
df_blog_empty = DataFrame(columns=['store_id','store_name','original_index','blog_tab'])

In [42]:
# 블로그 본문을 저장할 빈 DataFrame 생성
df_text = DataFrame(columns=['store_id', 'store_name', 'link', 'main text', 'original_index'])

In [43]:
# 플레이스의 업소별 ID를 담은 csv파일 DataFrame으로 변환
# 본인이 naver_place_count_total_2020-05-01_final.csv를 저장한 경로로 설정
df_place = \
DataFrame(pd.read_csv('./naver-place_csv/naver_place_count_total_2020-05-01_final.csv', sep=',', header=0, index_col=0))
df_place

,store_id,store_id_only_num,store_name,booking,receipt,blog,total
0,s1518997092,1518997092,준오헤어 노원사거리점,2015.0,3.0,15.0,2033.0
1,s1571970153,1571970153,차홍룸 홍대점,2009.0,1.0,10.0,2020.0
2,s1142939643,1142939643,차홍룸 용산센트럴점,1871.0,3.0,11.0,1885.0
3,s1481443590,1481443590,차홍룸 용산점,1541.0,2.0,11.0,1554.0
4,s1177346002,1177346002,차홍룸 양재점,1373.0,1.0,10.0,1384.0
...,...,...,...,...,...,...,...
2722,s38257322,38257322,바바헤어 대치점,1.0,1.0,2.0,4.0
2723,s1069634711,1069634711,정환헤어샵,1.0,1.0,2.0,4.0
2724,s20852762,20852762,머리짱헤어샵,1.0,2.0,1.0,4.0
2725,s36091089,36091089,살롱160,1.0,1.0,2.0,4.0


# 블로그 본문 크롤링 함수
## 위 블로그 리뷰 섬네일 크롤링 함수 모두 완료 후 진행해야 합니다!
#### 2020-05-13 (수) 이한울
---
- 헤드리스 적용
- 스레드 미구현
- **위 블로그 리뷰 섬네일 크롤링 함수 모두 완료 후 진행해야 합니다!**
---
#### 2020-05-14 (목) 이한울
---
- 스레드 구현 완료
- 본인이 저장한 CSV 파일의 경로와 이름명을 read_csv를 통해 DataFrame으로 변환한 뒤 실행하셔야 합니다.
- **반드시 위 리뷰 썸네일 크롤링 함수에서 본인의 범위까지 크롤링을 완료한 뒤 진행해야 합니다.**
- **함수가 다 실행된 후 CSV파일로 별도로 저장해야 합니다. (함수화 하지 않음)**

In [44]:
# 블로그리뷰 섬네일 저장 CSV 파일 읽어오기
# 경로 직접 설정하셔야 합니다.
csv_path = './place_review/2020-05-13_place_blog-thumb (0-362).csv'
df_blog_review = pd.read_csv(csv_path, sep=',', header=0, index_col=0)
df_blog_review

,store_id,store_name,title,writer,link,summary,blog_name,original_index
0,1571970153,차홍룸 홍대점,차홍룸 홍대점 by 정하은 디자이너 클리닉 룸클래식케어 후기 !,hyujing,https://m.blog.naver.com/hyujing/221896277248,거의 1년넘게 내 머리를 맡기고 있는 미용실 차홍룸 ! 아묻따 차홍룸에서 펌도 하고...,셀린's 라이프로그,1
1,1571970153,차홍룸 홍대점,"홍대미용실 차홍룸 룸클래식케어 보송해진 두피 "" 한민지디자이너 """,uareso_cute,https://m.blog.naver.com/uareso_cute/221849824368,홍대미용실 차홍룸 by. 한민지디자이너 예민해진 두피 룸클래식케어 보송보송! 푸~욱...,카페인의 일상 아카이브,1
2,1571970153,차홍룸 홍대점,차홍룸 홍대점 뿌리염색 완벽스 성공! 합정미용실 추천 (feat.정하은...,hyujing,https://m.blog.naver.com/hyujing/221813775267,"세상에서 제일 싫은것 중에 하나가, 염색후 1-2달 쯤 지나면 스멀스멀 올라오는 나...",셀린's 라이프로그,1
3,1571970153,차홍룸 홍대점,차홍룸 홍대점 레인펌 솔직후기! 히피펌 느낌♥ (ft. 한민지 디자이너),wlstngl,https://m.blog.naver.com/wlstngl/221818148879,안녕하세요 이웃님들! 새로미입니다 :D 저 이번에 지인짜 오랜만에 머리하고 왔더욤 ...,fashion＊beauty = 새롬c,1
4,1571970153,차홍룸 홍대점,단발펌이 고민이시라면 차홍룸 홍대미용실 (w.배희진디자이너쌤),leedan2,https://m.blog.naver.com/leedan2/221797129257,안녕하세요 단단입니다. 12월 말 경에 가족여행을 다녀왔는데요. 여행가는 겸사겸사 ...,오늘은 뭐 사먹지?,1
...,...,...,...,...,...,...,...,...
29289,36276109,제이드헤어,숙대 미용실 제이드헤어 굵은 웨이브펌 하고왔어요,wonywonyu,https://m.blog.naver.com/wonywonyu/220684840781,숙대 미용실 제이드헤어 봄봄봄이왔어요♥ 앞머리도 자르고 오랫만에 웨이브도 넣고싶어서...,더블리로그,362
29290,36276109,제이드헤어,숙대 미용실 제이드헤어 디자이너들 소개할게요,jadehair,https://m.blog.naver.com/jadehair/221262556729,안녕하세요^^ 오늘은 숙대 미용실 제이드헤어 디자이너 쌤들 소개를 해볼게요~~ 실은...,제이드의 뷰티라이프,362
29291,36276109,제이드헤어,숙대미용실 제이드헤어 재연쌤 짱인듯!,dduddu31,https://m.blog.naver.com/dduddu31/220465471913,숙대미용실 제이드헤어 (JADE HAIR) 친절하고 실력파 디자이너 쌤들이 많은듯!...,꿈꾸는 소녀의 뷰티&패션&여행,362
29292,36276109,제이드헤어,숙대 미용실 제이드헤어 소개할게요,jadehair,https://m.blog.naver.com/jadehair/220456067988,"안녕하세요 제이드에요 오늘은 KBS와 MBC 방송과 조선일보, 쎄씨 등~ 각종 매거...",제이드의 뷰티라이프,362


In [45]:
# 에러가 발생한 블로그를 저장할 DataFrame 생성하기
df_text_error = DataFrame(columns=['store_id','store_name','link','original_index'])

In [46]:
# requests status code가 200이 아닌 것들을 저장할 DataFrame 생성하기
df_requests_error = DataFrame(columns=['store_id', 'store_name', 'error_code', 'link', 'original_index'])

In [47]:
df_no_text = DataFrame(columns=['store_id','store_name','link','original_index'])

In [48]:
def sub_emoji(content):
    
    find_txt = (content.text).strip().replace('\n', '')
    p = re.compile(r'[\\u\w]{1,}')
    filtered_txt = p.findall(find_txt)
    main_txt = ' '.join(filtered_txt)
    
    return main_txt

In [49]:
def main_text_save(main_txt, store_id, store_name, url):
    
    global df_text
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_text= df_text.append({
          'main text' : main_txt
        , 'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_text

In [50]:
# 에러 발생 시 따로 저장할 함수 선언
def text_error_save(store_id, store_name, url):
    
    global df_text_error
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_text_error = df_text_error.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_text_error

In [51]:
def requests_error_save(store_id, store_name, error_code, url):
    
    global df_text_error
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_requests_error = df_requests_error.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'error_code' : error_code
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_requests_error

In [52]:
def blog_no_text(store_id, store_name, url):
    
    global df_no_text
    global df_place
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    
    df_no_text = df_no_text.append({
          'store_id' : store_id
        , 'store_name' : store_name
        , 'link' : url
        , 'original_index' : original_index
    }, ignore_index=True)
    
    return df_no_text   

In [53]:
# requests만 사용한 블로그 본문 크롤링 코드
# ==========================================================================
# 2020-05-12 (화) 이한울
# 본문을 긁어올 때에는 특별히 Action할 것이 없기 때문에
# requests를 통해서 크롤링하는 것이 효율적일 수도 있다고 생각하여 requests만 사용
# ==========================================================================

def text_save(url):
    
    global df_blog_review
    global df_text
    global df_place
    text_list = []
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36"} 
    store_id = df_blog_review[df_blog_review['link'] == url]['store_id'].iloc[0]
    store_name = df_blog_review[df_blog_review['link'] == url]['store_name'].iloc[0]
    
    original_index = \
    (df_place[df_place['store_id_only_num'] == store_id]['store_name'].index).item()
    url = url.replace('m.', '')
    time.sleep(1 + np.random.rand() + np.random.rand())
    
    print('\n', store_name, '의', url, '블로그의 본문 크롤링 중...\n')
    req = requests.get(url, headers)
    req_status = req.status_code
    time.sleep(1 + np.random.rand() + np.random.rand())
    
    if 'https://post.naver.com' in url:
        print('네이버 포스트 리뷰입니다. 크롤링에서 제외하고 다음 블로그를 크롤링합니다.')
        pass
    
    else:
        if req_status == 200:

            try:
                soup = BeautifulSoup(req.text, 'html.parser')
                bsb_main = soup.select('#mainFrame')

                if bsb_main:
                    for ii, blog in tqdm_notebook(enumerate(bsb_main)):
                        b_post = blog.attrs['src']
                        open_post = urlopen('http://blog.naver.com/' + b_post)
                        time.sleep(2.5)

                        b_html = BeautifulSoup(open_post, 'html.parser')
                        content = b_html.find("div", {"class": "se-main-container"})
                        main_txt = sub_emoji(content)
                        main_text_save(main_txt, store_id, store_name, url)
                        
                        # 스마트에디터3 타이틀 제거 임시 적용 (클래스가 다름)
                        if (content == None):
                            content = b_html.find("div", {"id": "postViewArea"})
                            main_txt = sub_emoji(content)
                            main_text_save(main_txt, store_id, store_name, url)
                            
                            if (content == None):
                                content = b_html.find("div", {"class": "se_component_wrap sect_dsc __se_component_area"})
                                main_txt = sub_emoji(content)
                                main_text_save(main_txt, store_id, store_name, url)
                                
                                if (content == None):
                                    content = b_html.find("div", {"class" : "se_component_wrap"})
                                    main_txt = sub_emoji(content)
                                    main_text_save(main_txt, store_id, store_name, url)
                                    
                                    if content == None:
                                        blog_no_text(store_id, store_name, url)
                                        print('잘못된 접근이거나 삭제된 게시물입니다.')
                                        print('다음 블로그 크롤링을 진행합니다.')
                                        pass
                                
                        print(url, '블로그의 본문 크롤링 성공!\n')

                else:
                    print('HTTP 에러 코드:',req.status_code)
                    text_error_save(store_id, store_name, url)

            except sce.NoSuchElementException:
                print('본문 크롤링 도중 에러가 발생했습니다.')
                text_error_save(store_id, store_name, url)

            except sce.NoSuchAttributeException:
                print('본문 크롤링 도중 에러가 발생했습니다.')
                text_error_save(store_id, store_name, url)

            except sce.ElementNotSelectableException:
                print('본문 크롤링 도중 에러가 발생했습니다.')
                text_error_save(store_id, store_name, url)

            except sce.WebDriverException:
                print('웹 드라이버 에러 발생')
                print('유니코드 변환 에러')
                text_error_save(store, store_name, url)
                
            except AttributeError:
                print('Attribute Error!')
                print('Error Code:', req_status)
                print('삭제된 게시물입니다.')
                
                blog_no_text(store_id, store_name, url)
                
        elif req_status in list(range(400, 506)):
            print('Requests Error가 발생했습니다. df_requests_error에', store_name + '의 정보를 저장합니다.')
            requests_error_save(store_id, store_name, req.status_code, url)
            pass
        
    print('현재까지 저장된 본문의 개수: ',len(df_text), '\n')
    time.sleep(1.25 + np.random.rand() + np.random.rand())
    
    
    req.close()
    return df_text

# 업소별 블로그 리뷰 본문 텍스트 크롤링 스레딩
#### 2020-05-14 (목) 이한울
---
- 앞선 플레이스 블로그 리뷰 썸네일 크롤링 함수에서 저장된 **df_blog_review** DataFrame이나 **CSV파일**로 따로 저장해두었다면 해당 CSV파일을 DataFrame으로 읽은 후 해당 DataFrame의 길이만큼 범위를 지정해주시면 됩니다.
- a와 end 값을 본인의 범위에 맞게 지정해주신 뒤 실행하시면 됩니다.
- 간혹 지정된 범위까지 크롤링을 하였음에도 계속해서 실행되어 뒷 부분까지 크롤링하는 상황이 발생하니 유의해주시기 바랍니다.
- **함수가 다 실행된 후 CSV파일로 별도로 저장해야 합니다. (함수화 하지 않음)**

In [54]:
# 크롬 웹 드라이버 헤드리스 설정 및 웹 보안 무시
options = webdriver.ChromeOptions()

prefs = {
    'profile.default_content_setting_values': {
          'cookies': 2, 'images': 2, 'plugins': 2
        , 'popups': 2, 'geolocation': 2, 'notifications': 2
        , 'auto_select_certificate': 2, 'fullscreen': 2, 'mouselock': 2
        , 'mixed_script': 2, 'media_stream': 2, 'media_stream_mic': 2
        , 'media_stream_camera': 2, 'protocol_handlers': 2, 'ppapi_broker': 2
        , 'automatic_downloads': 2, 'midi_sysex': 2, 'push_messaging': 2
        , 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2, 'protected_media_identifier': 2
        , 'app_banner': 2, 'site_engagement': 2, 'durable_storage': 2
    }
}

options.add_experimental_option('prefs', prefs)
options.add_argument('--disable-dev-shm-usage')
options.add_argument('User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36')
options.add_argument("Accept=text/html,application/xhtml+xml,application/xml;q=0.9,imgwebp,*/*;q=0.8")
options.add_argument("start-maximized")
options.add_argument('--no-sandbox')
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('headless')
options.add_argument("disable-gpu")
options.add_argument("--disable-web-security")

In [55]:
# 플레이스 블로그 본문 크롤링 함수의 스레딩 적용
# 2020-05-15 (금) df_blog_review[0:10004]까지 크롤링
# 이후 10004부터 Iter하면 됨
# =========================================================================
# 약 800 ~ 1000개의 리뷰 크롤링 이후에 AttributeError가 자주 발생한다.
# 하지만 해당 링크로 접속했을 시 블로그 글에 정상적으로 접속할 수 있다,
# 이는 자동화 수집을 방지하기 위해 네이버에서 의도적으로 막은 것으로 보인다.
# 따라서 한번 수집 시 800 ~ 1000개를 넘지 않도록 해야한다.
# =========================================================================
a = 10004 # 5759 #4583  # 4583 ~ 5564
start_no = 10004 # 5759
end = a + 5
end_df = 15004 # 5759
# 2020-05-15 (금) 블로그 썸네일 csv에서 10000개 블로그 크롤링 진행중
# =========================================================================



for i in tqdm_notebook(range(int(len(df_blog_review[start_no:end_df]) / 5))):
    
    print(str(a) + ' ~ ' + str(end - 1) + '의 블로그 본문 크롤링을 시작합니다.')
    with ThreadPool(5) as tp:
        # 스레드에 입력할 매개변수가 2개 이상일 경우 starmap을 사용한다.
        tp.map(
            text_save
            , [df_blog_review['link'][ii] for ii in range(a, end)]
        )
        
    a += 5
    end += 5
    time.sleep(0.5)
    
    if a > end_df:
        break

# ==========================================================================
# 정상적으로 본문 크롤링이 된 블로그 정보 CSV 파일로 저장
df_text.to_csv('./place_review/2020-05-16_place_blog-text (%i-%i).csv' %(start_no, end_df))

# 크롤링 도중 에러가 발생한 블로그 정보 CSV 파일로 저장
df_text_error.to_csv('./place_review/2020-05-16_place_blog-error (%i-%i).csv' %(start_no, end_df))

# Requests Error가 발생한 블로그 정보 CSV 파일로 저장
df_requests_error.to_csv('./place_review/2020-05-16_blog_requests-error (%i-%i).csv' %(start_no, end_df))

# 본문이 없거나, Attribute Error가 발생한 블로그 정보 CSV파일로 저장
df_no_text.to_csv('./place_review/2020-05-16_blog_no_text (%i-%i).csv' %(start_no, end_df))
# ==========================================================================

In [19]:
a = 10004 # 5759 #4583  # 4583 ~ 5564
start_no = 10004 # 5759
end_df = int(len(df_blog_review[start_no:15004]))
end_df

5000